In [ ]:
!pip install llama-index
!pip install sqlalchemy


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, insert, select

# Create SQLite in-memory database
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# Define the city_stats table
city_stats_table = Table(
    "city_stats",
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

# Create the table
metadata_obj.create_all(engine)

# Insert sample data
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]

with engine.begin() as connection:
    for row in rows:
        connection.execute(insert(city_stats_table).values(**row))

# Verify the data
stmt = select(city_stats_table)
with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)


In [ ]:
from llama_index.core import Settings

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import SQLDatabase

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",  # Use local Ollama model
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

# Connect LlamaIndex to the SQL database
sql_database = SQLDatabase(engine, include_tables=["city_stats"])


In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
    rows = connection.execute(text("SELECT city_name FROM city_stats")).fetchall()
    print(rows)


In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, 
    tables=["city_stats"], 
    llm=llm
)

# Query example
response = query_engine.query("Which city has the highest population?")
print(response)


In [ ]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine
from llama_index.core.objects import SQLTableNodeMapping, SQLTableSchema, ObjectIndex
from llama_index.core import VectorStoreIndex

# Map schema and build an object index
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [SQLTableSchema(table_name="city_stats")]

# Create an object index for table schemas
obj_index = ObjectIndex.from_objects(
    table_schema_objs, table_node_mapping, VectorStoreIndex
)

# Create a table retriever
table_retriever = obj_index.as_retriever(similarity_top_k=1)

# Initialize the SQLTableRetrieverQueryEngine with table_retriever
query_engine = SQLTableRetrieverQueryEngine(
    sql_database=sql_database,
    table_retriever=table_retriever  # Use the correct argument
)

# Query using the retriever query engine
response = query_engine.query("Which city has the highest population?")
print(response)


In [ ]:
from llama_index.core.retrievers import NLSQLRetriever

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

# Query using the retriever
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

# Display results
for result in results:
    print(result.metadata)


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)
print(response)
